In [ ]:
import pickle, glob
import numpy as np
import keras.utils as keras_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import BatchNormalization as BatchNorm
from keras.layers import Activation
from keras.callbacks import ModelCheckpoint, EarlyStopping
from music21 import converter, instrument, note, chord

In [2]:
# To generate a new list of notes to train against
# notes = []
# for file in glob.glob("./midi_songs/*.mid"):
#     midi = converter.parse(file)
#     notes_to_parse = None
#     parts = instrument.partitionByInstrument(midi)
#     if parts: # file has instrument parts
#         notes_to_parse = parts.parts[0].recurse()
#     else: # file has notes in a flat structure
#         notes_to_parse = midi.flat.notes
#     for element in notes_to_parse:
#         if isinstance(element, note.Note):
#             notes.append(str(element.pitch))
#         elif isinstance(element, chord.Chord):
#             notes.append('.'.join(str(n) for n in element.normalOrder))

# with open('data/music_notes.pkl', 'wb') as filepath:
#     pickle.dump(notes, filepath)
#     pickle.dump(pitchnames, filepath)
#     pickle.dump(n_vocab, filepath)

In [ ]:
with open('data/music_notes.pkl', 'rb') as filepath:
    notes = pickle.load(filepath)
    pitchnames = pickle.load(filepath)
    n_vocab = pickle.load(filepath)

In [4]:
sequence_length = 100
# get all pitch names
pitchnames = sorted(set(item for item in notes))
# create a dictionary to map pitches to integers
note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
network_input = []
network_output = []
# create input sequences and the corresponding outputs
for i in range(0, len(notes) - sequence_length, 1):
    sequence_in = notes[i:i + sequence_length]
    sequence_out = notes[i + sequence_length]
    network_input.append([note_to_int[char] for char in sequence_in])
    network_output.append(note_to_int[sequence_out])
n_patterns = len(network_input)
# reshape the input into a format compatible with LSTM layers
network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
# normalize input
network_input = network_input / float(n_vocab)
network_output = keras_utils.to_categorical(network_output)

In [ ]:
model = Sequential()
model.add(LSTM(
    512,
    input_shape=(network_input.shape[1], network_input.shape[2]),
    recurrent_dropout=0.3,
    return_sequences=True
))
model.add(LSTM(512, return_sequences=True, recurrent_dropout=0.3,))
model.add(LSTM(512))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(BatchNorm())
model.add(Dropout(0.3))
model.add(Dense(n_vocab))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

In [ ]:
checkpoint_filepath = "./models/model.keras"    
checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='loss',
    mode='min',
    save_best_only=True,
    verbose=0
)
early_stopping = EarlyStopping(
    monitor="loss",
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
)
callbacks_list = [early_stopping, checkpoint]     
model.fit(network_input, network_output, epochs=1000, batch_size=64, callbacks=callbacks_list)